In [21]:
# pip unistall ipywidgets
# pip install ipywidgets
# pip install tabulate
# pip install plotly 

In [22]:
import ipywidgets as w
from IPython.display import display, clear_output
import json

from utils import *

with open('data.txt', 'r') as f:
    parts = json.load(f)

In [23]:
####################################################################################################
# Іміджева складова
####################################################################################################
w1_widgets = [
    widget_typeBox('Материал корпуса', get_part_attr(parts, 'Body', 'material')),
    widget_flag('Портативность')
]
w1 = widget_group('Внешний вид', w1_widgets)

####################################################################################################
# Механічна складова
####################################################################################################
w2_widgets = [
    widget_typeBox('Тип двигателя', get_part_attr(parts, 'Engine', 'engine_type')),
#     widget_typeBox('Число двигателей', get_part_attr(parts, 'Engine', 'number')),
    widget_rangeBoxInt('Число двигателей', *get_boundaries(parts, 'Engine', 'number')),
    widget_typeBox('Тип движителя', get_part_attr(parts, 'Engine', 'propulsion_type')),
    
    widget_typeBox('Материал пропеллера', get_part_attr(parts, 'Propeller', 'material')),
#     widget_typeBox('Число пропеллеров', get_part_attr(parts, 'Propeller', 'number')),
    widget_rangeBoxInt('Число пропеллеров', *get_boundaries(parts, 'Propeller', 'number')),
]
w2 = widget_group('Механика', w2_widgets)

####################################################################################################
# Апаратна складова
####################################################################################################
w3_widgets = [
#     widget_typeBox('Наличие GPS', get_part_attr(parts, 'Software', 'GPS')),
    widget_flag('Наличие GPS'),
#     widget_typeBox('Наличие ИИ', get_part_attr(parts, 'Software', 'AI')),
    widget_flag('Наличие ИИ'),
    widget_typeBox('Производитель ', get_part_attr(parts, 'Software', 'manufacturer')),
    
#     widget_typeBox('Частота процессора, Гц', get_part_attr(parts, 'CPU', 'frequency')),
    widget_rangeBoxFloat('Частота процессора, Гц', *get_boundaries(parts, 'CPU', 'frequency')),
#     widget_typeBox('Количество ядер процессора', get_part_attr(parts, 'CPU', 'num_cores')),
    widget_rangeBoxInt('Количество ядер процессора', *get_boundaries(parts, 'CPU', 'num_cores')),
    
#     widget_typeBox('Дальность приема сигнала, м', get_part_attr(parts, 'Sensor', 'signal_acceptance_distance')),
    widget_rangeBoxFloat('Дальность приема сигнала, м', *get_boundaries(parts, 'Sensor', 'signal_acceptance_distance')),
    
#     widget_typeBox('Разрешение матрицы, Мп', get_part_attr(parts, 'Camera', 'resolution')),
    widget_rangeBoxFloat('Разрешение матрицы, Мп', *get_boundaries(parts, 'Camera', 'resolution')),
#     widget_typeBox('Угол обзора', get_part_attr(parts, 'Camera', 'angle')),
    widget_rangeBoxInt('Угол обзора', *get_boundaries(parts, 'Camera', 'angle')),
#     widget_typeBox('Дальность ИК подсветки, м', get_part_attr(parts, 'Camera', 'IR_illumination')),
    widget_rangeBoxInt('Дальность ИК подсветки, м', *get_boundaries(parts, 'Camera', 'IR_illumination')),
]
w3 = widget_group('Аппаратная составляющая', w3_widgets)

####################################################################################################
# Акумулятор
####################################################################################################
w4_widgets = [
#     widget_typeBox('Продолжительность полета, мин', get_part_attr(parts, 'Accumulator', 'duration')),
    widget_rangeBoxInt('Продолжительность полета, мин', *get_boundaries(parts, 'Accumulator', 'duration')),
#     widget_typeBox('Взлетная масса, кг', get_part_attr(parts, 'Accumulator', 'take_off_weight')),
    widget_rangeBoxInt('Взлетная масса, кг', *get_boundaries(parts, 'Accumulator', 'take_off_weight')),
#     widget_typeBox('Высота полета, м', get_part_attr(parts, 'Accumulator', 'height')),
    widget_rangeBoxInt('Высота полета, м', *get_boundaries(parts, 'Accumulator', 'height')),
]
w4 = widget_group('Аккумулятор', w4_widgets)

####################################################################################################
# Total
####################################################################################################
w5_widgets = [
#     widget_rangeBox('Общая стоимость', 0, 1000),
#     widget_rangeBox('Общий вес', 0, 1000)
     widget_rangeBoxInt('Стоимость, грн', 0, 10000),
     widget_rangeBoxInt('Общий вес, г', 0, 1000),
]
layout = w.Layout(width='480px', border='solid 0px', margin='10px',
                  justify_content='center', align_items='center')
w5 = widget_group('', w5_widgets, layout=layout)

In [24]:
def brute_force(parts, restriction):
    new_parts = []
    for part in get_keys(parts): #restriction.keys():
#         new_parts[part] = {}
        particular_parts = filter_parts(parts, part)
        for example in particular_parts:
            example_satisfies_conditions = True
            for attr in restriction[part].keys():
#                 print('{} {}: {}'.format(part, attr, restriction[part][attr]))
#                 print(example[part][attr])
                if isinstance(restriction[part][attr], str):
                    condition = example[part][attr] == restriction[part][attr]
                elif isinstance(restriction[part][attr], tuple):
                    min_value, max_value = restriction[part][attr]
                    condition = example[part][attr] >= min_value and example[part][attr] <= max_value
                elif isinstance(restriction[part][attr], bool):
                    condition = not restriction[part][attr] or (example[part][attr] and restriction[part][attr])
                if not condition:
                    example_satisfies_conditions = False
                    break
            if example_satisfies_conditions:
#                 print('{}'.format(restriction[part]))
                new_parts.append(example)
#                 print(example)
                    
#                 print('{} {}: {}'.format(part, attr, restriction[part][attr]))
    print(all(filter_parts(new_parts, part) != [] for part in get_keys(parts)))
    print(list(map(lambda part: '{}: {}'.format(part, filter_parts(new_parts, part) != []), get_keys(parts))))
    for part in new_parts:
        print(part)
#     map(new_parts)
    pass

def analyse(b, parts, window):
    clear_output()
    display(window)
    restriction = {}
    for key in get_keys(parts):
        restriction[key] = {}
        
    restriction['Body']['material'] = w1.children[1].value
    restriction['Body']['portability'] = w1.children[2].value

    restriction['Engine']['engine_type'] = w2.children[1].value
    restriction['Engine']['number'] = w2.children[2].value
    restriction['Engine']['propulsion_type'] = w2.children[3].value
    restriction['Propeller']['material'] = w2.children[4].value
    restriction['Propeller']['number'] = w2.children[5].value

    restriction['Software']['GPS'] = w3.children[1].value
    restriction['Software']['AI'] = w3.children[2].value
    restriction['Software']['manufacturer'] = w3.children[3].value
    restriction['CPU']['frequency'] = w3.children[4].value
    restriction['CPU']['num_cores'] = w3.children[5].value
    restriction['Sensor']['signal_acceptance_distance'] = w3.children[6].value
    restriction['Camera']['resolution'] = w3.children[7].value
    restriction['Camera']['angle'] = w3.children[8].value
    restriction['Camera']['IR_illumination'] = w3.children[9].value

    restriction['Accumulator']['duration'] = w4.children[1].value
    restriction['Accumulator']['take_off_weight'] = w4.children[2].value
    restriction['Accumulator']['height'] = w4.children[3].value
    
    restriction['Total'] = {}
    restriction['Total']['cost'] = w5.children[1].value
    restriction['Total']['weight'] = w5.children[2].value
    
#     print(restriction)
    brute_force(parts, restriction)

In [25]:
b1 = w.Button(
    description='Анализ',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

b1.on_click(lambda b: analyse(b, parts, window))

In [26]:
all_widgets = w.HBox([w.VBox([w1, w2, w5, b1], layout=bigGroup), w.VBox([w3, w4], layout=bigGroup)])

window = w.VBox([widget_header('Lab 5', bold=False, face=facefont),
                widget_header('Дрон. Требования к функциональным элементам', bold=False, size=5, face=facefont),
                all_widgets], layout=allLayout) 

display(window)

VBox(children=(HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Lab 5</font></div>"), HTML(value="<div align='center'><font size='5px' face='Arial' color='#0969A2'>Дрон. Требования к функциональным элементам</font></div>"), HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Внешний вид</font></div>"), Dropdown(description='Материал корпуса', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Кевлар', 'Углепластика', 'Экструдированный пенополистирол'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), Checkbox(value=False, description='Портативность', style=DescriptionStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механика</font></div>"), Dropdown(description='Тип двигателя', layout=Layout(justify_content='center', width='400px'), options=('Бесколлекторный электродвигатель', 'Коллекторный электродвигатель', 'Электродвигатель'), style=DescriptionStyle(description_width='initial'), value='Бесколлекторный электродвигатель'), FloatRangeSlider(value=(2.0, 6.0), continuous_update=False, description='Число двигателей', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), Dropdown(description='Тип движителя', layout=Layout(justify_content='center', width='400px'), options=('Винт', 'Реактивная тяга'), style=DescriptionStyle(description_width='initial'), value='Винт'), Dropdown(description='Материал пропеллера', layout=Layout(justify_content='center', width='400px'), options=('Волоконный композит', 'Карбон'), style=DescriptionStyle(description_width='initial'), value='Волоконный композит'), FloatRangeSlider(value=(2.0, 6.0), continuous_update=False, description='Число пропеллеров', layout=Layout(justify_content='flex-start', width='400px'), max=6.0, min=2.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), FloatRangeSlider(value=(0.0, 10000.0), continuous_update=False, description='Стоимость, грн', layout=Layout(justify_content='flex-start', width='400px'), max=10000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial')), FloatRangeSlider(value=(0.0, 1000.0), continuous_update=False, description='Общий вес, г', layout=Layout(justify_content='flex-start', width='400px'), max=1000.0, readout_format='d', step=1.0, style=SliderStyle(description_width='initial'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='10px', width='480px')), Button(description='Анализ', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Аппаратная составляющая</font></div>"), Checkbox(value=False, description='Наличие GPS', style=DescriptionStyle(description_width='initial')), Checkbox(value=False, description='Наличие ИИ', style=DescriptionStyle(description_width='initial')), Dropdown(description='Производитель ', layout=Layout(justify_content='center', width='400px'), options=('Agisoft', 'Harris Geospatial Solutions', 'Trimble'), style=DescriptionStyle(description_width='initial'), value='Agisoft'), FloatRangeSlider(value=(1.6, 3.0), continuous_update=False, description='Частота процессора, Гц', layout=Layout(justify_content='flex-start', width='400px'), max=3.0, min=1.6

False
['Sensor: True', 'Software: True', 'Propeller: True', 'CPU: True', 'Body: True', 'Engine: False', 'Accumulator: True', 'Camera: True']
{'Sensor': {'name': 'Namur', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 1}}
{'Sensor': {'name': 'VDI/VDE 3845', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 2}}
{'Sensor': {'name': 'DIN EN', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 3}}
{'Sensor': {'name': 'ATEX', 'price': 0, 'weight': 0, 'signal_acceptance_distance': 0.5}}
{'Software': {'name': 'Agisoft Photoscan', 'price': 360, 'weight': 0, 'GPS': True, 'AI': False, 'manufacturer': 'Agisoft'}}
{'Propeller': {'name': 'Phantom 3', 'price': 200, 'weight': 30, 'material': 'Волоконный композит', 'number': 2}}
{'CPU': {'name': 'TMS320C6678', 'price': 460, 'weight': 0, 'frequency': 1.8, 'num_cores': 6}}
{'CPU': {'name': 'DSP C66x', 'price': 980, 'weight': 0, 'frequency': 2.7, 'num_cores': 8}}
{'CPU': {'name': 'C6678', 'price': 145, 'weight': 0, 'frequency': 1.6, 'num_

### Ответ

In [9]:
####################################################################################################
# Іміджева складова
####################################################################################################
rw1_widgets = [
    widget_typeBox('Корпус', get_part_attr(parts, 'Body', 'name')),
]

rw1 = widget_group('Зовнішній вигляд', rw1_widgets)

####################################################################################################
# Механічна складова
####################################################################################################
rw2_widgets = [
    widget_typeBox('Двигуни', get_part_attr(parts, 'Engine', 'name')),
    
    widget_typeBox('Проперели', get_part_attr(parts, 'Propeller', 'name')),
]
rw2 = widget_group('Механіка', rw2_widgets)

####################################################################################################
# Апаратна складова
####################################################################################################
rw3_widgets = [
    widget_typeBox('Процесор', get_part_attr(parts, 'CPU', 'name')),
    
    widget_typeBox('Датчик зв\'язку', get_part_attr(parts, 'Sensor', 'name')),
    
    widget_typeBox('Програмне забезпечення', get_part_attr(parts, 'Software', 'name')),
    widget_typeBox('Камера', get_part_attr(parts, 'Camera', 'name')),
]
rw3 = widget_group('Апаратна складова', rw3_widgets)

####################################################################################################
# Акумулятор
####################################################################################################
rw4_widgets = [
    widget_typeBox('Акумулятор', get_part_attr(parts, 'Accumulator', 'name')),
]
rw4 = widget_group('Акумулятор', rw4_widgets)

####################################################################################################
# Total
####################################################################################################
totalLayout = w.Layout(width='200px', justify_content='flex-end')
rw5_widgets = [
      w.HBox([w.Label(value='Загальна вартість:', style=title_style),
              w.Label(value='', style=title_style)], layout=totalLayout),
      w.HBox([w.Label(value='Загальна вага:', style=title_style),
              w.Label(value='', style=title_style)], layout=totalLayout),
]
rw5 = widget_group('', rw5_widgets)

In [10]:
def get_price(body, engine, propellers, cpu, sensor, software, camera, accumulator):
    names = list(map(lambda part: list(part.values())[0]['name'], parts))
    price = 0
    for name in [body, engine, propellers, cpu, sensor, software, camera, accumulator]:
        assert name in names, '{} is not a valid name'.format(name)
        price += get_attr(list(filter(lambda part: list(part.values())[0]['name'] == name, parts)), 'price')[0]
    return price

def get_weight(body, engine, propellers, cpu, sensor, software, camera, accumulator):
    names = list(map(lambda part: list(part.values())[0]['name'], parts))
    weight = 0
    for name in [body, engine, propellers, cpu, sensor, software, camera, accumulator]:
        assert name in names, '{} is not a valid name'.format(name)
        weight += get_attr(list(filter(lambda part: list(part.values())[0]['name'] == name, parts)), 'weight')[0]
    return weight

def calculate(b):
    rw5.children[1].children[1].value = str(get_price(rw1.children[1].value,
                                                      rw2.children[1].value, rw2.children[2].value,
                                                      rw3.children[1].value, rw3.children[2].value,
                                                      rw3.children[3].value, rw3.children[4].value,
                                                      rw4.children[1].value)) + ' грн'
    rw5.children[2].children[1].value = str(get_weight(rw1.children[1].value,
                                                       rw2.children[1].value, rw2.children[2].value,
                                                       rw3.children[1].value, rw3.children[2].value,
                                                       rw3.children[3].value, rw3.children[4].value,
                                                       rw4.children[1].value)) + ' г'

In [11]:
rb1 = w.Button(
    description='Прорахувати',
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
)

rb1.on_click(calculate)

all_widgets = w.HBox([w.VBox([rw1, rw2, rw5], layout=bigGroup), w.VBox([rw3, rw4, rb1], layout=bigGroup)],
                     layout=allLayout)

display(widget_header('Lab 5'))
display(widget_header('Drone', bold=False))
display(all_widgets)

HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'><b>Lab 5</b></font></div>")

HTML(value="<div align='center'><font size='6px' face='Arial' color='#0969A2'>Drone</font></div>")

HBox(children=(VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Зовнішній вигляд</font></div>"), Dropdown(description='Корпус', layout=Layout(justify_content='center', width='400px'), options=('AgDrone', 'Hubsan_H501S', 'Raytheon', 'Тахион'), style=DescriptionStyle(description_width='initial'), value='AgDrone')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Механіка</font></div>"), Dropdown(description='Двигуни', layout=Layout(justify_content='center', width='400px'), options=('DJI SPARK', 'Syma X5SW\xa0', 'Syma X8HW', 'WL Toys WL-Q282Jw'), style=DescriptionStyle(description_width='initial'), value='DJI SPARK'), Dropdown(description='Проперели', layout=Layout(justify_content='center', width='400px'), options=('AEE Propeller', 'DJI 9450 CW', 'DJI Inspire 2', 'Phantom 3'), style=DescriptionStyle(description_width='initial'), value='AEE Propeller')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'></font></div>"), HBox(children=(Label(value='Загальна вартість:', style=DescriptionStyle(description_width='initial')), Label(value='', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px')), HBox(children=(Label(value='Загальна вага:', style=DescriptionStyle(description_width='initial')), Label(value='', style=DescriptionStyle(description_width='initial'))), layout=Layout(justify_content='flex-end', width='200px'))), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px'))), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px')), VBox(children=(VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Апаратна складова</font></div>"), Dropdown(description='Процесор', layout=Layout(justify_content='center', width='400px'), options=('C6678', 'DSP C66x', 'DSP А871', 'TMS320C6678'), style=DescriptionStyle(description_width='initial'), value='C6678'), Dropdown(description="Датчик зв'язку", layout=Layout(justify_content='center', width='400px'), options=('ATEX', 'DIN EN', 'Namur', 'VDI/VDE 3845'), style=DescriptionStyle(description_width='initial'), value='ATEX'), Dropdown(description='Програмне забезпечення', layout=Layout(justify_content='center', width='400px'), options=('Agisoft Photoscan', 'ENVI OneButton', 'PHOTOMOD UAS', 'UASMaster'), style=DescriptionStyle(description_width='initial'), value='Agisoft Photoscan'), Dropdown(description='Камера', layout=Layout(justify_content='center', width='400px'), options=('AVT 3300AUXSD', 'AVT 6600AUXSDWF', 'AVT AHDCH 108', 'JIMI JH012'), style=DescriptionStyle(description_width='initial'), value='AVT 3300AUXSD')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), VBox(children=(HTML(value="<div align='center'><font size='4px' face='Arial' color='#0969A2'>Акумулятор</font></div>"), Dropdown(description='Акумулятор', layout=Layout(justify_content='center', width='400px'), options=('Aladin', 'LiPo', 'Luftera', 'Skywalker X8'), style=DescriptionStyle(description_width='initial'), value='Aladin')), layout=Layout(align_items='center', border='solid 0px', justify_content='center', margin='0px', width='450px')), Button(description='Прорахувати', style=ButtonStyle())), layout=Layout(align_items='center', border='solid 0px', height='530px', justify_content='space-around', padding='20px', width='500px'))), layout=Layout(align_items='stretch', border='solid 1px', justify_content='center', padding='20px'))

In [9]:
# get_price(body, engine, propellers, cpu, sensor, software, camera, accumulator)

In [56]:
get_weight(w1.children[1].value,
           w2.children[1].value, w2.children[2].value,
           w3.children[1].value, w3.children[2].value,
           w3.children[3].value, w3.children[4].value,
           w4.children[1].value)

835

In [20]:
d

{'Accumulator': {},
 'Body': {},
 'CPU': {},
 'Camera': {},
 'Engine': {},
 'Propeller': {},
 'Sensor': {},
 'Software': {}}